In [23]:
import pandas as pd
from itertools import cycle
import shutil
import os

In [16]:
caminho = os.path.join('..', '..', 'db_softtek') 
path_arq = os.path.abspath(caminho) 

In [17]:
tbl_train_path = os.path.join(path_arq, 'modelo', 'Modelo_score_train.xlsx')
tbl_test_path = os.path.join(path_arq, 'cubo_anl', 'tbl_full_analitica.xlsx')

tbl_train = pd.read_excel(tbl_train_path)
analistas_df = tbl_train

In [18]:
# Load the test data (tbl_full_analitica.xlsx)
tbl_test = pd.read_excel(tbl_test_path)

# Filter out necessary columns from the test data (novos chamados)
novos_chamados_df = tbl_test[['CHAMADO', 'DT_ABERTURA_CHAMADO', 'TIPO_CHAMADO', 'MODULO_CHAMADO', 'COMPLEXIDADE']].drop_duplicates(keep='first')


In [ ]:
# Agrupar analistas por módulo e ordenar por rank
analistas_por_modulo = analistas_df.sort_values(by=['MODULO_CHAMADO', 'rank']).groupby('MODULO_CHAMADO')

# Criar iteradores rotativos para cada módulo
iteradores = {modulo: cycle(grupo['IS'].tolist()) for modulo, grupo in analistas_por_modulo}

# Função para distribuir chamados
def distribuir_chamados(novos_chamados_df, iteradores):
    alocacao = []
    for _, chamado in novos_chamados_df.iterrows():
        modulo = chamado['MODULO_CHAMADO']
        if modulo in iteradores:
            analista = next(iteradores[modulo])
            alocacao.append(analista)
        else:
            alocacao.append(None)  # Caso não haja analistas para o módulo
    return alocacao

# Distribuir os novos chamados
novos_chamados_df['IS'] = distribuir_chamados(novos_chamados_df[['TIPO_CHAMADO','MODULO_CHAMADO','COMPLEXIDADE']], iteradores)

# Exibir a distribuição
novos_chamados_df.sort_values(['TIPO_CHAMADO','MODULO_CHAMADO']).ffill()
novos_chamados_df['ANOMES_ABERTURA'] = novos_chamados_df['DT_ABERTURA_CHAMADO'].str[0:4] + novos_chamados_df['DT_ABERTURA_CHAMADO'].str[5:7]


# Criando Pivot Table para os gráficos:

pivot_REDIST = novos_chamados_df.pivot_table(
    index='ANOMES_ABERTURA',
    columns='COMPLEXIDADE',
    values='CHAMADO',
    aggfunc='count'
).reset_index().fillna(0)

pivot_REDIST.columns.name = None

pivot_REDIST = pivot_REDIST[pivot_REDIST['ANOMES_ABERTURA'] >= '202401']
pivot_REDIST

In [ ]:
# Output
output_folder_modelo = os.path.join(path_arq, 'modelo')
os.makedirs(output_folder_modelo, exist_ok=True)

output_excel_path = os.path.join(output_folder_modelo, 'alocacao_recursos.xlsx')
pivot_REDIST.to_excel(output_excel_path, index=False)
print(f"EXCEL salvo: {output_excel_path}")

output_json_path = os.path.join(output_folder_modelo, 'cubo3_alocacao_recursos.json')
pivot_REDIST.reset_index().drop(columns='index').to_json(output_json_path, orient='records', indent=4)
print(f"JSON salvo: {output_json_path}")

In [ ]:
# Definir caminhos
caminho_base = os.path.abspath(os.path.join('..', '..', 'db_softtek'))
caminho_cubo_anl = os.path.join(caminho_base, 'cubo_anl')
caminho_modelo = os.path.join(caminho_base, 'modelo')
caminho_dados_frontend = os.path.abspath(os.path.join(os.path.dirname(__file__), '..', '..', '..', 'frontend', 'data'))

# Garantir que o diretório de destino exista
if not os.path.exists(caminho_dados_frontend):
    print(f"Criando diretório '{caminho_dados_frontend}'\n")
    os.makedirs(caminho_dados_frontend)
else:
    print(f"'{caminho_dados_frontend}' já existe\n")

# Lista de arquivos JSON para copiar
arquivos_json = [
    'aberturas_encerrados.json',
    'leadtime_mes_a_mes.json',
    'encerrados_complexidade.json',
    'senioridade_custo.json',
    'encerramento_senioridade.json'
]

# Copiar arquivos JSON do diretório cubo_anl para a pasta de dados do frontend
for nome_arquivo in arquivos_json:
    origem = os.path.join(caminho_cubo_anl, nome_arquivo)
    destino = os.path.join(caminho_dados_frontend, nome_arquivo)
    if os.path.exists(origem):
        try:
            shutil.copy(origem, destino)
            print(f"'{nome_arquivo}' salvo: '{destino}'\n")
        except Exception as e:
            print(f"Erro ao copiar '{nome_arquivo}': {e}\n")
    else:
        print(f"Arquivo '{nome_arquivo}' não encontrado em {caminho_cubo_anl}\n")

# Copiar e renomear o arquivo JSON específico
origem_cubo3 = os.path.join(caminho_modelo, 'cubo3_alocacao_recursos.json')
destino_cubo3 = os.path.join(caminho_dados_frontend, 'acompanhamento_da_redistribuicao.json')

if os.path.exists(origem_cubo3):
    try:
        shutil.copy(origem_cubo3, destino_cubo3)
        print(f"'cubo3_alocacao_recursos.json' salvo: '{destino_cubo3}'\n")
    except Exception as e:
        print(f"Erro ao copiar 'cubo3_alocacao_recursos.json': {e}\n")
else:
    print(f"Arquivo 'cubo3_alocacao_recursos.json' não encontrado em {caminho_modelo}\n")

print('Todos os arquivos processados.\n')

In [ ]:
# Salva o CSV com a alocação dos novos chamados para email_notif.ipynb
csv_path = os.path.join(output_folder_modelo, 'novos_chamados.csv')
novos_chamados_df.to_csv(csv_path, index=False)
print(f"CSV salvo: {csv_path}")